References:
1. https://medium.com/analytics-vidhya/deep-dream-visualizing-the-features-learnt-by-convolutional-networks-in-pytorch-b7296ae3b7f
2. https://towardsdatascience.com/how-to-visualize-convolutional-features-in-40-lines-of-code-70b7d87b0030

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch
import torchvision
from torchvision import transforms
# from ShapeData import train_loader, valid_loader

In [2]:
# to load the model from parent directory
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('../neural networks/Shapes'))))

In [3]:
'''
model = models.resnet18(pretrained=True)
print(model)
'''
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(".."))))
# importing
from CNNmodel import ConvNet

# load model
model = ConvNet()
model.load_state_dict(torch.load("..\\model.pth"))

<All keys matched successfully>

In [4]:
# freeze model parameterd during training
for param in model.parameters():
    param.requires_grad_(False)

print(model)

ConvNet(
  (conv1): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=320000, out_features=2, bias=True)
)


In [6]:
activation = {} # dictionary to store the activation of a layer

def create_hook(name):
    def hook(m, i, o):
        # copy the output of the given layer
        activation[name] = o
 
    return hook

In [8]:
# register a forward hook for output layer
model.fc.register_forward_hook(create_hook('output_layer'))

In [10]:
# transform
transform = transforms.Compose ([
            transforms.ConvertImageDtype(torch.float32),
            transforms.Normalize ([0.5], [0.5]) #0-1 to [-1,1], formula (x-mean)/std
        ])

In [11]:
# initialize 28*28 tensor, upscale it for interpretability

Height = 28
Width = 28
# generate a numpy array with random values
img = np.single(np.random.uniform(0,1, (3, Height, Width)))
# convert to a torch tensor, normalize, set the requires_grad_ flag
im_tensor = transform(torch.from_numpy(img)).requires_grad_(True)